## Imports

Begin by installing the shap library:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install shap

Now import all necessary dependencies:

In [ ]:
import shap
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# Set a seed for reproducibility
seed = 42
tf.random.set_seed(seed)

## Train a CNN model

For this lab you will use the [fashion MNIST](https://keras.io/api/datasets/fashion_mnist/) dataset. Load it and pre-process the data before feeding it into the model:


In [ ]:
# Download the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
y_train_true, y_random_labels = y_train, np.random.permutation(y_train)
# Reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(10000, 28, 28, 1).astype("float32") / 255
random_ = False

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

# Create the model with the corresponding inputs and outputs
model = keras.Model(inputs=inputs, outputs=outputs, name="CNN")

# Compile the model
model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer=keras.optimizers.Adam(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )
if(random_):
  # Train it!
  history = model.fit(x_train, y_random_labels, epochs=100, validation_data=(x_test, y_test), batch_size=128)
else:
  history = model.fit(x_train, y_train_true, epochs=50, validation_data=(x_test, y_test), batch_size=128)

In [ ]:
# Name each one of the classes
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Save an example for each category in a dict
images_dict = dict()
for i in range(len(y_train_true)):
    label = y_train_true[i]
    if label not in images_dict:
        images_dict[label] = x_train[i].reshape((28, 28))

# Function to plot images
def plot_categories(images):
    fig, axes = plt.subplots(1, 11, figsize=(16, 15))
    axes = axes.flatten()

    # Plot an empty canvas
    ax = axes[0]
    dummy_array = np.array([[[0, 0, 0, 0]]], dtype='uint8')
    ax.set_title("reference")
    ax.set_axis_off()
    ax.imshow(dummy_array, interpolation='nearest')

    # Plot an image for every category
    for k, v in sorted(images.items()):
        ax = axes[k+1]
        ax.imshow(v, cmap=plt.cm.binary)
        ax.set_title(f"{class_names[k]}")
        ax.set_axis_off()

    plt.tight_layout()
    plt.show()

# Use the function to plot
plot_categories(images_dict)

In [ ]:
# Select 5000 random samples from x_test
background = x_test[np.random.choice(x_test.shape[0], 1500, replace=False)]

# Use DeepExplainer to explain predictions of the model
e = shap.DeepExplainer(model, background)

In [ ]:
# Save an example of each class from the test set
x_test_dict = dict()
for i, l in enumerate(y_test):
  if len(x_test_dict)==10:
    break
  if l not in x_test_dict.keys():
    x_test_dict[l] = x_test[i]

# Convert to list preserving order of classes
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]

# Convert to tensor
x_test_each_class = np.asarray(x_test_each_class)

# Print shape of tensor
print(f"x_test_each_class tensor has shape: {x_test_each_class.shape}")

In [ ]:
# Compute predictions
predictions = model.predict(x_test_each_class)

# Apply argmax to get predicted class
np.argmax(predictions, axis=1)

In [ ]:
shap_values = e.shap_values(x_test_each_class)

In [ ]:
# Plot reference column
plot_categories(images_dict)

# Print an empty line to separate the two plots
print()

# Plot shap values
shap.image_plot(shap_values, -x_test_each_class)

In [ ]:
import numpy as np
if(random_):
  # Assuming shap_values_list is a list of numpy arrays, each with shape (1, 28, 28, 1)
  shap_values_dict = {f'shap_value_{i}': arr.squeeze() for i, arr in enumerate(shap_values)}

  # Save the SHAP values
  np.savez('/content/drive/MyDrive/fmnist_rand/shap_values_true_fmnist.npz', **shap_values_dict)
else:
    # Assuming shap_values_list is a list of numpy arrays, each with shape (1, 28, 28, 1)
  shap_values_dict = {f'shap_value_{i}': arr.squeeze() for i, arr in enumerate(shap_values)}

  # Save the SHAP values
  np.savez('/content/drive/MyDrive/fmnist/shap_values_true_fmnist.npz', **shap_values_dict)


In [ ]:
test_images = x_test_each_class[0]
test_images.shape

In [ ]:
x_test[0].shape

In [ ]:
test_image = x_test_each_class[0].reshape(1, 28, 28, 1)


e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_image)

In [ ]:
test_image.shape

In [ ]:
# Plot reference column
plot_categories(images_dict)
shap.image_plot(shap_values, test_image.reshape(1,28,28,1))

In [ ]:
import numpy as np

if(random_):
  # Assuming shap_values_list is a list of numpy arrays, each with shape (1, 28, 28, 1)
  shap_values_dict_single_dress = {f'shap_value_{i}': arr.squeeze() for i, arr in enumerate(shap_values)}

  # Save the SHAP values
  np.savez('/content/drive/MyDrive/fmnist_rand/shap_values_rand_fmnist_single_tshirt.npz', **shap_values_dict_single_dress)
else:
  # Assuming shap_values_list is a list of numpy arrays, each with shape (1, 28, 28, 1)
  shap_values_dict_single_dress = {f'shap_value_{i}': arr.squeeze() for i, arr in enumerate(shap_values)}

  # Save the SHAP values
  np.savez('/content/drive/MyDrive/fmnist/shap_values_fmnist_single_tshirt.npz', **shap_values_dict_single_dress)



In [ ]:
len(shap_values)

In [ ]:
shap_values[0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Calculate the overall maximum and minimum values across all images
max_intensity = np.max([np.max(np.abs(image.squeeze(axis=0))) for image in shap_values])
min_intensity = -max_intensity

# Create a grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

# Iterate through each image in shap_values and plot it
for i, ax in enumerate(axes.flat):
    # Extract the 3D array from the list
    image = shap_values[i].squeeze(axis=0)  # Squeeze to remove the first dimension (1)

    # Plot the image with a custom colormap
    im = ax.imshow(image, cmap='RdBu_r', vmin=min_intensity+0.02, vmax=max_intensity-0.02)
    ax.axis('off')  # Hide axis labels

# Add a colorbar to show the correspondence between color and intensity
cbar = fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.03, pad=0.1)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Create a grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

# Iterate through each image in shap_values and plot it
for i, ax in enumerate(axes.flat):
    # Extract the 3D array from the list
    image = shap_values[i].squeeze(axis=0)  # Squeeze to remove the first dimension (1)

    # Calculate the maximum absolute intensity value for setting color scale
    max_abs_intensity = np.max(np.abs(image))

    # Plot the image with a custom colormap
    im = ax.imshow(image, cmap='RdBu_r', vmin=-max_abs_intensity, vmax=max_abs_intensity)
    ax.axis('off')  # Hide axis labels

# Add a colorbar to show the correspondence between color and intensity
cbar = fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.03, pad=0.1)

# Show the plot
plt.show()


In [ ]:
import pickle

if(random_):
  # Save the entire model to a HDF5 file
  model.save("/content/drive/MyDrive/fmnist_rand/your_model.keras")

  # Save the history object to a file using pickle
  with open("/content/drive/MyDrive/fmnist_rand/training_history.pkl", "wb") as file:
      pickle.dump(history.history, file)
else:
  # Save the entire model to a HDF5 file
  model.save("/content/drive/MyDrive/fmnist/your_model.keras")

  # Save the history object to a file using pickle
  with open("/content/drive/MyDrive/fmnist/training_history.pkl", "wb") as file:
      pickle.dump(history.history, file)


In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras

visualkeras.layered_view(model, legend = True)

In [ ]:
model.summary()